In [ ]:
%pylab notebook
from __future__ import print_function
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sys
pd.options.display.max_rows = 100

In [ ]:
import charistools.find_best_model as fbm

In [ ]:
%cd /Users/brodzik/mjb/charis/calibrations/summit/SY_Naryn_calibration_2strikes
%ls -las



In [ ]:
filename = 'Naryn_01.out'
drainage = 'Naryn'
nstrikes = 2
#filename = 'Hunza_calibration-negRainfall-ET.out'
#drainage = "IN_Hunza_at_DainyorBridge"
#nstrikes = 3
cal = fbm.get_calibration_stats(filename, verbose=1)
cal["stats"]

In [ ]:
df = cal["stats"]

In [ ]:
#df.dtypes

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(8,6))

fig.suptitle(
    "%s Best=%s \n(%d strikes, RMSE=%.3f $km^3$, Voldiff=%.3f%%, numModels=%d)" % (
        drainage, df.index[0], 
        nstrikes,
        df["Monthly_rmse_km3"].iloc[0], 
        df["Annual_voldiff_pcent"].iloc[0],
        len(df.index)),
    fontsize=12)

surfaces = ["ice", "snow"]
markers =  ["^", "o"]
for i, surface in enumerate(surfaces):
    
    row = i
    min_ddf = "min_%s_ddf" % surface
    max_ddf = "max_%s_ddf" % surface
    # Upper Left: RMSE as function of min/max snow ddf 
    col = 0
    rmse_plot = ax[row, col].scatter(df[min_ddf],
                             df[max_ddf],
                             c=df["Monthly_rmse_km3"],
                             cmap="Greens",
                             edgecolor="",
                             marker=markers[i])
    ax[row, col].plot([df[min_ddf].iloc[0]],
                      [df[max_ddf].iloc[0]],
                      marker="x",
                      color="red")
    fig.colorbar(rmse_plot, ax=ax[row,col])
    ax[row, col].set_title('Monthly RMSE ($km^3$)')
    ax[row, col].set_xlabel(min_ddf)
    ax[row, col].set_ylabel(max_ddf)

    # Middle: Annual volume difference
    col = 1
    vmax = np.max(df["Abs_voldiff_pcent"])
    vmin = -1.0 * vmax
    voldiff_plot = ax[row, col].scatter(df[min_ddf],
                             df[max_ddf],
                             c=df["Annual_voldiff_pcent"],
                             cmap="BrBG_r",
                             vmin=vmin,
                             vmax=vmax,
                             edgecolor="",
                             marker=markers[i])
    ax[row, col].plot([df[min_ddf].iloc[0]],
                      [df[max_ddf].iloc[0]],
                      marker="x",
                      color="red")
    fig.colorbar(voldiff_plot, ax=ax[row,col])
    ax[row, col].set_title('Annual vol diff (%)')
    ax[row, col].set_xlabel(min_ddf)
    ax[row, col].set_ylabel(max_ddf)

    # Right: Combined Z score
    col = 2
    z_plot = ax[row, col].scatter(df[min_ddf],
                             df[max_ddf],
                             c=df["z"],
                             cmap="Greys",
                             vmin=0.0,
                             edgecolor="",
                             marker=markers[i])
    ax[row, col].plot([df[min_ddf].iloc[0]],
                      [df[max_ddf].iloc[0]],
                      marker="x",
                      color="red")
    fig.colorbar(z_plot, ax=ax[row,col])
    ax[row, col].set_title('Z')
    ax[row, col].set_xlabel(min_ddf)
    ax[row, col].set_ylabel(max_ddf)


fig.tight_layout()
fig.subplots_adjust(top=0.86)

In [ ]:
len(df.index)

